### MANDATORY ASSIGNMENT 2

In [8]:
from sklearn import datasets

In [9]:
import numpy as np

In [10]:
iris = datasets.load_iris()

In [11]:
X = iris.data
Y = iris.target

#### Task 1) data exploration

In [12]:
len(X)

150

In [13]:
print(X.shape, Y.shape)

(150, 4) (150,)


In [14]:
print(np.min(X), np.max(X))
print(np.min(Y), np.max(Y))

0.1 7.9
0 2


In [15]:
from sklearn.preprocessing import MinMaxScaler
from qiskit import QuantumCircuit, transpile, assemble
from qiskit_aer import Aer, AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import Parameter
from qiskit_algorithms.optimizers import SPSA

from sklearn.metrics import log_loss # loss function
from sklearn.metrics import accuracy_score # accuracy
import time
import pickle

In [16]:
scaler = MinMaxScaler(feature_range=(0, np.pi)) # since we are using angle encoding, we need to scale from 0 to pi
X = scaler.fit_transform(X)

In [63]:
class QuantumMachineLearning:
    def __init__(self, X_train, y_train, X_val, y_val, num_qubits = 4, num_layers = 3):
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        self.num_qubits = num_qubits
        self.num_layers = num_layers
        self.num_classes = len(set(y_train))
        self.rng = np.random.default_rng(42)
        self.initial_parameters = self.rng.uniform(0, np.pi, self.num_qubits * self.num_layers)

        self.base_circuit = self._create_base_circuit()
        self.backend = AerSimulator(method = 'statevector')
    
    def angle_encoding(self, qc, data_point):
        [qc.rx(data_point[qubit], qubit) for qubit in range(self.num_qubits)]
        return qc

    def real_amplitudes(self, qc, parameters):
        param_indices = np.arange(len(parameters)).reshape(self.num_layers, self.num_qubits)
        
        for layer in range(self.num_layers):
            [qc.ry(parameters[param_indices[layer, qubit]], qubit) 
             for qubit in range(self.num_qubits)]
            qc.barrier()
            
            [qc.cx(qubit, qubit+1) for qubit in range(self.num_qubits-1)]
            qc.barrier()

        return qc

    def _create_base_circuit(self, encoding = 'angle', circuit = 'real_amplitudes'):
        qc = QuantumCircuit(self.num_qubits)
        if encoding == 'angle':
            qc = self.angle_encoding(qc, np.array([Parameter(f"theta_{i}") for i in range(self.num_qubits)]))
        
        if circuit == 'real_amplitudes':
            qc = self.real_amplitudes(qc ,np.array([Parameter(f"theta_{i}{j}")
                                                     for i in range(self.num_qubits)
                                                     for j in range(self.num_layers)]))
        
        return qc


    def prepare_circuit(self, data_point, params):
        qc = self.base_circuit.copy()
        qc.assign_parameters(np.concatenate([data_point, params]), inplace = True)
        
        return qc
   
    def run_circuit(self, data_point, params, shots = 100):
        qc = self.prepare_circuit(data_point, params)     
        
        qc.measure_all()

        tqc = transpile(qc, self.backend)

        job = self.backend.run(tqc, shots=shots)
        result = job.result()
        counts = result.get_counts(qc)

        return counts

    
    def data_decoding(self, output):
        return int(output, 2) % self.num_classes
    
    def loss_function(self, updated_params, val = False, shots = 100):
        X = self.X_train if not val else self.X_val
        y = self.y_train if not val else self.y_val

        def process_counts(x):
            counts = self.run_circuit(x, updated_params, shots = shots)

            count_classes = {x : 0 for x in range(self.num_classes)}
            [count_classes.update({self.data_decoding(output): 
                                 count_classes.get(self.data_decoding(output), 0) + count/shots}) 
             for output, count in counts.items()]
        
            return [count_classes[x] for x in range(self.num_classes)]        
            
        predicted_probabilites = np.array([process_counts(x) for x in X])
        logloss = log_loss(y, predicted_probabilites)

        #print(f"Parameters: {updated_params} loss: {logloss}")
        return logloss

    def SPSA_optimize(self, maxiter = 50):
        optimizer = SPSA(maxiter=maxiter)
        # Optimize the parameters
        optimized = optimizer.minimize(fun=self.loss_function, x0=self.initial_parameters)

        print("Optimized Parameters:", optimized.x)
        print("Minimum Loss:", optimized.fun)
        self.optimized_params = optimized.x
        self.min_loss = optimized.fun
    

    def gradient(self, params, epsilon = 0.2):
        size = len(params)
        plus = np.broadcast_to(params, (size, size)) + np.eye(size) * epsilon
        minus = np.broadcast_to(params, (size, size)) - np.eye(size) * epsilon
        L_plus = np.array([self.loss_function(p) for p in plus])
        L_minus = np.array([self.loss_function(p) for p in minus])

        return (L_plus - L_minus) / (2 * epsilon)
    
    
    def run_gradient_descent(self, learning_rate = 0.1, maxiter = 50, shots = 100):
        self.all_epochs = np.empty((0, 3))
        current_point = self.initial_parameters
        parameter_storage = np.empty((0, len(current_point)))
        early_stopping = False
        for epoch in range(maxiter):
            start = time.time()

            gradients = self.gradient(current_point)
            current_point = [current_point[j] - learning_rate * gradients[j] for j in range(len(gradients))]

            training_loss  = self.loss_function(current_point, shots= shots)
            validation_loss = self.loss_function(current_point, val = True, shots= shots)
            parameter_storage = np.vstack([parameter_storage, current_point])

            end = time.time()
            elapsed_time = end - start
            self.all_epochs = np.vstack([self.all_epochs, [training_loss, validation_loss, elapsed_time]])

            if epoch >= 10:
                val_losses = self.all_epochs[:,1]
                if np.mean(val_losses[-5:]) > np.mean(val_losses[-10:]): # if last 5 validation losses are greater than last 10, break
                    best_index = np.argmin(val_losses)
                    current_point = parameter_storage[best_index]
                    print(f"""Early Stopping at epoch {epoch}, 
                        Training Loss: {self.all_epochs[:,0][best_index]}, 
                        Validation Loss: {val_losses[best_index]},""")
                    early_stopping = True
                    break

            print(f"Epoch {epoch} Training Loss: {training_loss}, Validation Loss: {validation_loss}, Time: {elapsed_time}")
       
        current_point = [float(p) for p in current_point]
        loss = training_loss if not early_stopping else self.all_epochs[:,0][best_index]
        print("Optimized Parameters:", current_point, "Loss:", loss)
        self.optimized_params = current_point  
        self.min_loss = loss
    
    def predict(self, data_point): #paramteres must be optimized before prediction
        prediction_shots = 100000 # more shots as the circuit is only run once
        counts = self.run_circuit(data_point, self.optimized_params, shots = prediction_shots)

        predicted_probabilites = {x : 0 for x in range(self.num_classes)}
            
        # Decode each measurement outcome and aggregate probabilites for each class
        for output, count in counts.items():
            class_num = self.data_decoding(output)
            predicted_probabilites[class_num] += count / prediction_shots
        
        
        # Determine the predicted class by choosing the class with the highest probability
        predicted_class = max(predicted_probabilites, key=predicted_probabilites.get)
        
        return predicted_class

    def predict_dataset(self, X):
        return [self.predict(data_point) for data_point in X]
    
    def performance(self, y_test, X_test):
        self.accuracy = accuracy_score(y_test, self.predict_dataset(X_test))
        return self.accuracy

    def save(self):
        with open('model_data.pkl', 'wb') as f:
            pickle.dump({
                'optimized_params': self.optimized_params,
                'min_loss': self.min_loss,
                'accuracy': self.accuracy,
                'all_epochs': self.all_epochs
            }, f)

    
    
    

In [64]:
from sklearn.model_selection import train_test_split

In [65]:
X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.3, random_state=42) # 70% training 
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42) # 15% validation, 15% testing

In [66]:
object1 = QuantumMachineLearning(X_train, y_train, X_val, y_val, num_layers=5)

object1.run_gradient_descent(maxiter = 100, learning_rate=0.7)

Epoch 0 Training Loss: 0.8501950318097843, Validation Loss: 0.641097166047387, Time: 88.16916513442993
Epoch 1 Training Loss: 0.7398126483387472, Validation Loss: 0.5259067556102622, Time: 94.84738826751709
Epoch 2 Training Loss: 0.6781531016125129, Validation Loss: 0.5211175717499514, Time: 97.23017311096191
Epoch 3 Training Loss: 0.6115834101843195, Validation Loss: 0.4284135345393627, Time: 105.76415014266968
Epoch 4 Training Loss: 0.5953945315447589, Validation Loss: 0.43530839982679925, Time: 112.65723872184753
Epoch 5 Training Loss: 0.5428719825703259, Validation Loss: 0.39427647939522414, Time: 105.55296492576599
Epoch 6 Training Loss: 0.5466089500165255, Validation Loss: 0.39502905414139433, Time: 106.61407399177551
Epoch 7 Training Loss: 0.5392090471497898, Validation Loss: 0.4132839440290765, Time: 113.57430720329285
Epoch 8 Training Loss: 0.5260870927056398, Validation Loss: 0.40624139447883284, Time: 116.4483380317688
Epoch 9 Training Loss: 0.5392109552286685, Validation Lo

In [67]:
object1.performance(y_test, X_test)

1.0

In [68]:
object1.save()

In [69]:
with open ('model_data.pkl', 'rb') as f:
    data = pickle.load(f)
data

{'optimized_params': [2.3004669842083616,
  0.23847854375513025,
  3.1454281409730793,
  1.6169980091053295,
  -0.013246819270140611,
  3.522625181927975,
  2.080617176657913,
  2.848168320103692,
  -0.008944667770985293,
  1.2734739436003266,
  1.7404200996009178,
  3.422557890377305,
  1.8572519924103577,
  2.4890481970960607,
  1.1649847177572783,
  1.725377925129511,
  1.4825428589302658,
  0.44015431003665956,
  2.657233455778988,
  1.9471040484051299],
 'min_loss': np.float64(0.4500287954854215),
 'accuracy': 1.0,
 'all_epochs': array([[8.50195032e-01, 6.41097166e-01, 8.81691651e+01],
        [7.39812648e-01, 5.25906756e-01, 9.48473883e+01],
        [6.78153102e-01, 5.21117572e-01, 9.72301731e+01],
        [6.11583410e-01, 4.28413535e-01, 1.05764150e+02],
        [5.95394532e-01, 4.35308400e-01, 1.12657239e+02],
        [5.42871983e-01, 3.94276479e-01, 1.05552965e+02],
        [5.46608950e-01, 3.95029054e-01, 1.06614074e+02],
        [5.39209047e-01, 4.13283944e-01, 1.13574307e+0